In [1]:
api_key = "AIzbSyDfByrNpVlLGjpXrvgE0882wP04R6hdn78"

In [2]:
import langchain
from langchain.llms import GooglePalm
from langchain.prompts import PromptTemplate

In [3]:
from pypdf import PdfReader
import pandas as pd
import json

In [4]:
cols = ["Sold By", "Billing Address", "State/UT Code", "Shipping Address", "PAN No",
    "GST Registration No", "Place of supply", "Place of delivery", "Order Number", "Order Date", "Invoice Number", "Invoice Date", 
    "Invoice Details", "Description", "Unit Price", "Qty", "Net Amount", "Tax Rate", "Tax Type", "Tax Amount", "Total Amount",
    "Amount in Words", "Payment Transaction ID", "Date & Time", "Invoice Value", "Mode of Payment"]

In [5]:
def main(file_names):
    
    # Extracting COntent From PDF
    def get_pdf_text(pdf_doc):
        text = ""
        pdf_reader = PdfReader(pdf_doc)
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

    # Feeding extracted content from pdf to llm to give output as fields name and it's values
    def extracted_data(invoice):
        
        # tuning LLm to fullfill our purpose using prompt template
        template = """Extract all the following values : Sold By, Billing Address, State/UT Code, Shipping Address, PAN No,
        GST Registration No, Place of supply, Place of delivery, Order Number, Order Date, Invoice Number, Invoice Date, Invoice Details, 
        Description, Unit Price, Qty, Net Amount, Tax Rate, Tax Type, Tax Amount, Total Amount, Amount in Words, Payment Transaction ID, 
        Date & Time, Invoice Value, Mode of Payment: {pages}

        Expected output:{{'Invoice number.': '100','Description': 'Nutella','Qty': '2','Date': '1/1/2024','Unit price': '100.00','Total Amount': '1000.00'}}
        """

        prompt_template = PromptTemplate(input_variables=["pages"], template=template)

        llm = GooglePalm(google_api_key=api_key,temperature=0.7)
        full_response=llm(prompt_template.format(pages=invoice))


        return full_response
    
    # this is for first file so that we can create one instance of dictionary and append values of other files to this dictionary
    data = get_pdf_text(file_names[0])    # applying get_pdf_text()
    edata = extracted_data(data)          # applying extracted_data()
    edata = edata.replace("\n","")        # replacing space character
    edata = edata.replace("'",'"')        # replacing sigle quote to double quote of fields so that we can deserialize it using json
    dic = json.loads(edata)               # deserializing (converting to dictionary)
    lists = {key: [value] for key, value in dic.items()} # converting values of fields to list so that we can append values from other files
    
    # this is for other files
    for file in file_names[1:]:
        data = get_pdf_text(file)
        edata = extracted_data(data)
        edata = edata.replace("\n","")
        edata = edata.replace("'",'"')
        dic = json.loads(edata)

        for key,value in dic.items():
            lists[key].append(value)
            
    common_df = pd.DataFrame(lists)  # converting dictionary to dataframe
    sub_df = common_df[["Description","Total Amount", "Tax Rate","Tax Type","Tax Amount"]]
    
    # converting pdf/dataframe to excel
    with pd.ExcelWriter('output.xlsx') as writer:
        common_df.to_excel(writer, sheet_name="sheet1", index=False)
        sub_df.to_excel(writer, sheet_name="sheet2", index=False)
    
    return common_df 
        
    
  

In [6]:
file_names = ["invoice.pdf","invoice1.pdf","invoice2.pdf","invoice3.pdf"]

In [9]:
df = main(file_names)

In [10]:
pd.read_excel('output.xlsx',sheet_name='sheet1')

,Sold By,Billing Address,State/UT Code,Shipping Address,PAN No,GST Registration No,Place of supply,Place of delivery,Order Number,Order Date,...,Net Amount,Tax Rate,Tax Type,Tax Amount,Total Amount,Amount in Words,Payment Transaction ID,Date & Time,Invoice Value,Mode of Payment
0,"ASSPL-Amazon Seller Services Pvt. Ltd., ARIPL-...",ACEFOUR ACCESSORIES PRIVATE LIMITED * Survey N...,29,Ashish Srivastava Ashish SrivastavaBlock A Fla...,AAVCA3022G,29AAVCA3022G1ZL,KARNATAKA,KARNATAKA,405-8002819-1037154,04.12.2023,...,915.26,9%,CGST,82.37,"1,080.00",One Thousand Eighty only,i93r1KfscTn1hMQ8aSo7mod0eYSciu6cjuZ,"04/12/2023, 00:18:37hrs","1,080.00",UPI
1,RETAILEZ PRIVATE LIMITED,Ashish SrivastavaBlock A Flat 201 Sonestaa Mea...,29,Ashish SrivastavaBlock A Flat 201 Sonestaa Mea...,AALCR3173P,09AALCR3173P1ZL,KARNATAKA,KARNATAKA,405-8002819-1037154,04.12.2023,...,550.00,18%,IGST,99.00,649.00,Six Hundred Forty-nine only,i93r1KfscTn1hMQ8aSo7mod0eYSciu6cjuZ,"04/12/2023, 00:18:37hrs",649.00,UPI
2,Appario Retail Private Ltd,"Adarsh SrivastavaFlat 201, block A, Sonestaa m...",29,Adarsh Srivastava Ashish SrivastavaBlock a fla...,AALCA0171E,29AALCA0171E1ZV,KARNATAKA,KARNATAKA,405-1392927-6043509,04.12.2023,...,575.42,9%,CGST,51.79,679.00,Six Hundred Seventy-nine only,i93r1KfkNGoSVA2mmeejghcqpErOnz5trdp,"04/12/2023, 00:03:18hrs",679.00,UPI
3,Amazon Retail India Private Limited,"Adarsh Srivastava Flat 201, block A, Sonestaa ...",29,"Adarsh Srivastava Adarsh Srivastava Flat 201, ...",AAPCA6346P,29AAPCA6346P1ZR,KARNATAKA,KARNATAKA,406-4260355-4309154,09.12.2023,...,"98.20, 575.90, 71.42","6%, 6%, 2.5%","CGST, CGST, SGST","5.90, 34.55, 1.79",830.00,Eight Hundred Thirty only,NaN,NaN,830.00,NaN


In [11]:
pd.read_excel('output.xlsx',sheet_name='sheet2')

,Description,Total Amount,Tax Rate,Tax Type,Tax Amount
0,uppercase Medium 17L 1700EBP1 School Backpack ...,"1,080.00",9%,CGST,82.37
1,Gear Campus8 25L Medium Water Resistant School...,649.00,18%,IGST,99.00
2,SanDisk Ultra UHS I 64GB SD Card 140MB/s for D...,679.00,9%,CGST,51.79
3,Nutrela Soya Mini Chunks 220 g | B013SAX1NG ( ...,830.00,"6%, 6%, 2.5%","CGST, CGST, SGST","5.90, 34.55, 1.79"


# 